In [1]:
import glob
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt

SFID = 64
ws = 16

def getKernelO(img, x, y, ws):
    x = int(np.round(x))
    y = int(np.round(y))
    counter = 0
    h,w = img.shape
    o = 0.0
    for i in range(y+int(-ws/2),y+int(ws/2)+1):
        for j in range(x+int(-ws/2),x+int(ws/2)+1):
            if i>=0 and i<h and j>=0 and j<w:
                o += (img[i,j]/255.0)
                counter += 1
    return o/counter            

In [2]:
files_folder = "AM09_LWIR_V000_1/"
correspondence_files = glob.glob(files_folder+"corr*")

In [3]:
file_relation_container = {}
max_KFID = SFID
for i in range(len(correspondence_files)):
    file_path_split = correspondence_files[i].rpartition('/')
    if "NKF" in file_path_split[2]:
        # Process NKFs
        parts = file_path_split[2][7:-4].split('_')
        BKFID = int(parts[0])
        NKFID = int(parts[1])
        HKFID = int(parts[2])
        if BKFID < SFID or HKFID < SFID or NKFID==0:
            continue
        if BKFID not in file_relation_container:
            file_relation_container[BKFID] = {"HKFs":[], "FNKFs":{}}
        if NKFID not in file_relation_container[BKFID]["FNKFs"]:
            file_relation_container[BKFID]["FNKFs"][NKFID] = []
        file_relation_container[BKFID]["FNKFs"][NKFID].append(HKFID)
    else:
        parts = file_path_split[2][4:-4].split('_')
        CKFID = int(parts[0])
        HKFID = int(parts[1])
        if CKFID < SFID+1 or HKFID < SFID or HKFID==CKFID:
            continue
        if CKFID not in file_relation_container:
            file_relation_container[CKFID] = {"HKFs":[], "FNKFs":{}}
        file_relation_container[CKFID]["HKFs"].append(HKFID)
        max_KFID = max(max_KFID, CKFID)

In [ ]:
# %matplotlib notebook
# fig, axs = plt.subplots(2)
# plt.ion()
# fig.show()
# fig.canvas.draw()
frame_counter = 1 # KF SFID is the first frame
KF_2_frame_counter = {SFID:0}
ds_file = open("Correspondences_KF.txt",'w+')
for i in range(SFID, max_KFID+1):
    ### Process Keyframes
    CKFID = i
    if len(file_relation_container[CKFID]["HKFs"]) > 0:
        KF_2_frame_counter[CKFID] = frame_counter
        img_current = cv2.imread(files_folder+"imNKF"+str(CKFID)+'_0.png',2)
        should_increment = False
        for HKFID in file_relation_container[CKFID]["HKFs"]:
            try:
                corr_file = open(files_folder+"corr"+str(CKFID)+"_"+str(HKFID)+".txt",'r')
                img_history = cv2.imread(files_folder+"imNKF"+str(HKFID)+'_0.png',2)
#                 axs[0].clear()
#                 axs[1].clear()
#                 axs[0].imshow(img_history,cmap='gray')
#                 axs[1].imshow(img_current,cmap='gray')
                for line in corr_file:
                    parts = line.split(',')
                    o_current = getKernelO(img_current,float(parts[2]),float(parts[3]),ws)
                    o_history = getKernelO(img_history,float(parts[0]),float(parts[1]),ws)
                    csv_string = str(frame_counter)+','+str(frame_counter-KF_2_frame_counter[HKFID])+','+str(o_history)+','+str(o_current)+',1,'+line
                    ds_file.write(csv_string)
#                     color = tuple(np.random.random((3,)))
#                     axs[0].scatter([int(float(parts[0]))],[int(float(parts[1]))],color=color)
#                     axs[1].scatter([int(float(parts[2]))],[int(float(parts[3]))],color=color)
#                 fig.canvas.draw()
#                 time.sleep(3)
                should_increment = True
            except:
                print("Skipping File", "corr"+str(CKFID)+"_"+str(HKFID)+".txt")
        if should_increment:
            print("Processing frame",frame_counter)
            frame_counter+=1
        else:
            print("Big ======== Error =========================")

    ### Process Non-Keyframes
    sorted_list_nkfs = list(file_relation_container[CKFID]["FNKFs"].keys())
    sorted_list_nkfs.sort()
    for j in sorted_list_nkfs:
        NKFID = j
        should_increment = False
        img_current = cv2.imread(files_folder+"imNKF"+str(CKFID)+'_'+str(NKFID)+'.png',2)
        for HKFID in file_relation_container[CKFID]["FNKFs"][j]:
            try:
                corr_file = open(files_folder+"corrNKF"
                                 +str(CKFID)+'_'+str(NKFID)+'_'+str(HKFID)+".txt", 'r').readlines()
                img_history = cv2.imread(files_folder+"imNKF"+str(HKFID)+'_0.png',2)
                for line in corr_file:
                    parts = line.split(',')
                    o_current = getKernelO(img_current,float(parts[2]),float(parts[3]),ws)
                    o_history = getKernelO(img_history,float(parts[0]),float(parts[1]),ws)
                    csv_string = str(KF_2_frame_counter[CKFID]+NKFID)+','+str(KF_2_frame_counter[CKFID]+NKFID-KF_2_frame_counter[HKFID])+','+str(o_history)+','+str(o_current)+',0,'+line
                    ds_file.write(csv_string)
                should_increment = True
            except:
                print("Skipping File", "corrNKF"+str(CKFID)+'_'+str(NKFID)+'_'+str(HKFID)+".txt")
        if should_increment:
            frame_counter+=1
        else:
            print("Big ======== Error 2 =========================")
ds_file.close()

In [5]:
max_KFID = 731